# Scoring your trained model

In the cell below, please load your model into `model`. Also if you used an image size for your input images that *isn't* 224x224, you'll need to set `image_size` to the size you used. The scoring code assumes square input images.

For example, this is how I loaded in my checkpoint:

```python
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

class FFClassifier(nn.Module):
    
    def __init__(self, in_features, hidden_features, 
                       out_features, drop_prob=0.1):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(p=drop_prob)
        
    def forward(self, x):
        x = self.drop(F.relu(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    
    model = models.vgg16(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = FFClassifier(25088, checkpoint['hidden'], 102)
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

model = load_checkpoint('/home/workspace/classifier.pt')
```

Your exact code here will depend on how you defined your network in the project. Make sure you use the absolute path to your checkpoint which should have been uploaded to the `/home/workspace` directory.

Run the cell, then after loading the data, press "Test Code" below. This can take a few minutes or more depending on the size of your network. Your model needs  to reach **at least 20% accuracy** on the test set to be recorded.

In [1]:

import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

In [2]:
model = models.vgg19(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(25088, 4096),
                                 nn.ReLU(),
                                 nn.Dropout(0.4),
                                 nn.Linear(4096, 102),
                                 nn.LogSoftmax(dim=1))
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.4)
  (3): Linear(in_features=4096, out_features=102, bias=True)
  (4): LogSoftmax()
)

In [3]:
criterion = nn.CrossEntropyLoss()

#Specify Optimizer: SGD with LR 0.01
import torch.optim as optim
optimizer = optim.SGD(model.classifier.parameters(), lr = 0.01)

In [4]:
model.load_state_dict(torch.load('umodel.pt', map_location=lambda storage, loc: storage),strict=False)
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [9]:
import torch
import torch.nn as nn
from torchvision import models

#Load your model to this variable
model = models.vgg19(pretrained = True) 
model.classifier = nn.Sequential(nn.Linear(25088, 4096),
                                 nn.ReLU(),
                                 nn.Dropout(0.4),
                                 nn.Linear(4096, 102),
                                 nn.LogSoftmax(dim=1))

#Add Loss Function (Categorical Cross Entropy)
criterion = nn.CrossEntropyLoss()

#Specify Optimizer: SGD with LR 0.01
import torch.optim as optim
optimizer = optim.SGD(model.classifier.parameters(), lr = 0.01) ## ResNet, Inception may differ Slightly
# USE model.parameters() in ResNet or Inception

#Load Model State Dictionary from Downloaded model.pt file
Model_State_Path = '/home/workspace/umodel.pt'
# Convert GPU Model State Dictionary to CPU based
model.load_state_dict(torch.load(Model_State_Path, map_location=lambda storage, loc: storage),strict=False)
model.eval() #Don't Forget to add this Line

#### DEFAULT THINGS FOR UDACITY WORKSPACE #######
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224 #For Inception v3 it is 299
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]